In [1]:
import pandas as pd
from datetime import datetime
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import mainUtils
from statistics import mean
from itertools import chain 

In [2]:
!python yolov7/detectTrack.py --weights yolov7/best.pt --no-trace --view-img --source yolov7/testImg/carsVideo.mp4 --save-with-object-id

Namespace(weights=['yolov7/best.pt'], download=False, source='yolov7/testImg/carsVideo.mp4', img_size=640, conf_thres=0.25, iou_thres=0.45, device='', view_img=True, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='object_tracking', exist_ok=False, no_trace=True, colored_trk=False, save_bbox_dim=False, save_with_object_id=True)
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
video 1/1 (1/2704) c:\Users\abdel\OneDrive - The British University in Egypt\Desktop\Uni\Graduation Project\Code\yolov7\testImg\carsVideo.mp4: 8 Cars, Done. (34.3ms) Inference, (70.6ms) NMS
video 1/1 (2/2704) c:\Users\abdel\OneDrive - The British University in Egypt\Desktop\Uni\Graduation Project\Code\yolov7\testImg\carsVideo.mp4: 8 Cars, Done. (65.9ms) Inference, (3.0ms) NMS
video 1/1 (3/2704) c:\Users\abdel\OneDrive - The British University in Egypt\Desktop\Uni\Gra

YOLOR  2023-3-27 torch 1.11.0+cu113 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 6143.6875MB)

C:\Users\abdel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36519530 parameters, 6194944 gradients, 103.3 GFLOPS


In [3]:
folder_path='runs\detect\object_tracking2\labels'
batch_folder='data/batches2'
batchLabels_folder='data/batchLabels2'

In [4]:
mainUtils.files_into_batches(folder_path, batch_folder)

In [5]:
mainUtils.process_txt(batch_folder, batchLabels_folder)

In [7]:
mainUtils.VCount_CSV(batchLabels_folder, 'data/count2.csv')

In [8]:
mainUtils.VType_csv(batchLabels_folder, 'data/type2.csv')

In [9]:
data=pd.read_csv('data/count2.csv')

In [10]:
data

,Time,Lane 1 Flow (Veh/5 Minutes),# Lane Points,% Observed
0,2023-06-10 06:58:29,52,1,100
1,2023-06-10 06:58:29,25,1,100
2,2023-06-10 06:58:29,19,1,100
3,2023-06-10 06:58:29,18,1,100
4,2023-06-10 06:58:29,15,1,100
5,2023-06-10 06:58:29,52,1,100
6,2023-06-10 06:58:29,32,1,100
7,2023-06-10 06:58:29,30,1,100
8,2023-06-10 06:58:29,35,1,100
9,2023-06-10 06:58:29,38,1,100


In [11]:
model=load_model('TrafficFlowPred\RMSmodel.h5')

scaler = MinMaxScaler(feature_range=(0, 1)).fit(data['Lane 1 Flow (Veh/5 Minutes)'].values.reshape(-1, 1))
flow = scaler.transform(data['Lane 1 Flow (Veh/5 Minutes)'].values.reshape(-1, 1)).reshape(1, -1)[0]

test = []
for i in range(12, len(flow)):
    test.append(flow[i - 12: i + 1])

test = np.array(test)

X_test = test[:, :-1]
y_test = test[:, -1]


pred = model.predict(X_test)
pred = scaler.inverse_transform(pred.reshape(-1, 1)).reshape(1, -1)[0]
y_preds = []
y_preds.append(pred)
print(y_preds)
average=mean(list(chain.from_iterable(y_preds)))
if((average)>=25):
    print("The Street is congested, We recommend taking another way")
else:
    print("The Street is not congested, We recommend using it more")

1/1 [==============================] - 3s 3s/step
[array([29.38342], dtype=float32)]
The Street is not congested, We recommend using it more
